# TME 04 - IAMSI

### Exercice 2

#### Question 1. Exprimer en fonction de ne et nj le nombre de variables propositionnelles utilisées

Le nombre de variables propositionelles utilisées est nJ * nE * (nE - 1).

En effet, chaque jour, chaque équipe peut affronter toutes les autres équipes (sauf elle-même evidemment).

In [1]:
def nbVar(nE, nJ):
    return nJ*nE*(nE-1)

#### Question 2. Ecrire une fonction de codage qui prend en argument ne, nj , j, x, y, et qui renvoie k

In [2]:
def getK(nE, nJ, j, x, y):
    return j*nE**2 + x*nE + y + 1

#### Question 3. Ecrire une fonction de décodage qui retrouve j, x et y à partir de k, ne.

In [3]:
def decodage(k, nE):
    k = k-1
    y = k%nE
    k = k//nE
    x = k%nE
    j = k//nE
    
    return j, x, y

### Exercice 3 - Génération d’un planning de matchs

#### Question 1. Contraintes de cardinalité.
Il s’agit ici de se doter de quelques fonctions pour faciliter l’encodage du problème avec des contraintes de cardinalité ”au plus 1” et ”au moins 1”. Pour la contrainte ”au plus 1”, on considère dans cet exercice un encodage par paires.

##### 1. Ecrire une fonction qui prend en argument une liste d’entiers et renvoie une clause, au format DIMACS, correspondant à la contrainte ”au moins une de ces variables est vraie” (Σivi ≥ 1).

In [4]:
def atLeastOne(liste):
    res = ""
    for elem in liste:
        res+= str(elem) +" "
    res += "0\n"
    return res 

In [5]:
print(atLeastOne([1, -2, 7, 5]))

1 -2 7 5 0



##### 2. Ecrire une fonction qui prend en argument une liste d’entiers et renvoie une clause, au format DIMACS, correspondant à la contrainte ”au plus une de ces variables est vraie” (Σivi >= 1).

In [6]:
def atMostOne(liste):
    n = len(liste)
    res = ""
    for i in range(n):
        for j in range(i, n):
            if i != j:
                res += str(-liste[i]) + " " + str(-liste[j]) + " 0\n"
                
    return res


In [7]:
print(atMostOne([1,-2, 3, 4]))

-1 2 0
-1 -3 0
-1 -4 0
2 -3 0
2 -4 0
-3 -4 0



#### Question 2. Traduction du problème.
On s’attache maintenant à traduire le problème en une formule logique propositionnelle, en passant éventuellement par l’intermédiaire de contraintes de cardinalite.

##### 1. Traduire la contrainte C1 ”chaque équipe ne peut jouer plus d’un match par jour” en un ensemble de contraintes de cardinalité.

##### 2. Ecrire une fonction encoderC1 qui génère ces contraintes pour ne et nj donné.

In [8]:
def decoderListe(liste, nE):
    for elem in liste:
        j,x,y = decodage(elem, nE)
        print(elem, " : jour", j,", equipe", x, " vs. equipe", y)
    

In [10]:
def encoderC1(nE, nJ):
    #print("encoderC1")
    res = ""
    for j in range(nJ):
        #print("jour = ", j)
        for x in range(nE):
            #print("\tx =", x)
            liste = []
            for y in range(nE):
                if x != y:
                    liste.append(getK(nE, nJ, j, x, y))
                    liste.append(getK(nE, nJ, j, y, x))

                    
            #decoderListe(liste, nE)
            res += atMostOne(liste)
            
    return res

encoderC1
nb de clauses attendues =  72
nb de clauses obtenues =  72


##### 3. Indiquer le nombre de contraintes et de clauses générées pour 3 équipes sur 4 jours et expliciter ces contraintes.

#### LE CAS GÉNÉRAL:

Pour chaque jour (nJ jours), chaque équipe (nE equipes) ne peut jouer ne peut jouer qu'au plus UN match (que ce soit à domicile ou à l'exterieur).
Donc il y'a $nJ*nE$ contraintes.

Pour chaque contrainte (equipe et jour fixé), on verifie que cette equipe a au maximum un match ce jour là (atMostOne). Donc, on vérifie que parmi les variables représentant cette equipe à ce jour (il y en a $2(nE-1)= 2nE-2$) , il n'y en ai qu'une maximum qui soit verifiée.

Cela donne donc $(((2nE-2)*(2nE-3))/2)$ clauses par contrainte.
On a $nJ*nE*(((2nE-2)*(2nE-3))/2)$ clauses au total pour les contraintes de types C1.

#### CAS AVEC 3 ÉQUIPES ET 4 JOURS:
On a donc 3*4 = 12 contraintes qui génèreront en tout 72 clauses.

In [24]:
def nbClausesC1(nE, nJ):
    return int(nJ * nE * ((2*nE-2) * (2*nE -3))/2)

In [25]:
nE = 3
nJ = 4
c = encoderC1(nE, nJ)
print("nb de clauses attendues = ", nbClausesC1(nE, nJ))
print("nb de clauses obtenues = ", c.count("0\n"))

#print(c)

encoderC1
nb de clauses attendues =  72
nb de clauses obtenues =  72


##### 4. Traduire la contrainte C2 ”Sur la durée du championnat, chaque équipe doit rencontrer l’ensemble des autres équipes une fois à domicile et une fois à l’extérieur, soit exactement 2 matchs par équipe adverse.” en un ensemble de contraintes de cardinalités.

##### 5. Ecrire une fonction encoderC2 qui génère ces contraintes pour ne et nj donné.


In [34]:
def encoderC2(nE, nJ):
    #print("encoderC2")
    res = ""
    l1 = []
    l2 = []
    for equipe in range(nE):
        #print("\nequipe =", equipe)
        
        for y in range(equipe, nE):
            l1 = []
            l2 = []
            if equipe != y:
                #print("\ty =", y)
                for j in range(nJ):
                    l1.append(getK(nE, nJ, j, equipe, y))
                    l2.append(getK(nE, nJ, j, y, equipe))
                #print("un match a domicile avec equipe", y)
                #decoderListe(l1, nE)
                #print("un match à l'exterieur avec equipe", y)
                #decoderListe(l2, nE)
            
                res += atLeastOne(l1) + atMostOne(l1)
                res += atLeastOne(l2) + atMostOne(l2)
    return res

##### 6. Indiquer le nombre de contraintes et de clauses générées pour 3 équipes sur 4 jours et expliciter ces contraintes.


#### LE CAS GÉNÉRAL:
Il y a deux contraintes par couples d'équipes differentes, afin que chaque équipe ait exactement 1 match a domicile et 1 match a l'exterieur avec toutes les autres equipes
Cela donne $((nE(nE-1))/2) * 2 = nE*(nE-1)$ contraintes.

Chaque contrainte génère 1 clause pour le atLeastOne et $((nJ(nJ-1))/2)$ pour le atMostOne

Donc le nombre total de clauses est : $nE(nE-1) * (1+((nJ(nJ-1))/2))$


#### CAS AVEC 3 ÉQUIPES ET 4 JOURS:
On a donc 6 contraintes qui génereront 6*(1+6) = 42 clauses

In [28]:
def nbClausesC2(nE, nJ):
    return int(nE*(nE-1) * (1+((nJ*(nJ-1))/2)))


In [29]:
nE = 3
nJ = 4
c = encoderC2(nE, nJ)
print("nb de clauses attendues =", nbClausesC2(nE, nJ))
print("nb de clauses obtenues =", c.count("0\n"))


encoderC2
nb de clauses attendues = 42
nb de clauses obtenues = 42


##### 7. Ecrire un programme encoder qui encode toutes les contraintes C1 et C2 pour ne et nj donné.

In [30]:
def encoder(nE, nJ):
    return "c contraintes de types C1\n"+encoderC1(nE, nJ) +"c contraintes de types C2\n"+ encoderC2(nE, nJ)

In [31]:
def nbClauses(nE, nJ):
    return nbClausesC1(nE, nJ) + nbClausesC2(nE, nJ)

In [32]:
def creerCNF(nE, nJ, chemin):

    f = open(chemin, "w")
    title = "c title\n"
    nl = "c\n"
    intro = "p cnf " + str(nbVar(nE, nJ)) +" "+ str(nbClauses(nE, nJ))+ "\n"
    c = encoder(nE,nJ)
    f.writelines([title, nl, intro, c])
    
    

In [33]:
nE = 3
nJ = 4
chemin = "./cnf/doc"
creerCNF(nE, nJ, chemin+"1.cnf")

encoderC1
encoderC2


#### Question 3. Utilisation du solveur.
Utiliser glucose sur la CNF générée à la question précédente et vérifier la première solution proposée pour 3 équipes sur 4 jours. Qu’est-il nécessaire d’ajouter aux deux contraintes C1 et C2 ?


#### Question 4. Décodage.
Ecrire une fonction ou un programme decoder qui prend pour argument un fichier contenant la sortie de l’appel à glucose (plus éventuellement nj et ne) et qui traduit le modèle rendu en une solution du problème de planning des matchs affichée lisiblement. On peut faire appel à un fichier extérieur donnant le nom des équipes : une par ligne dans leur ordre de numérotation.


#### Question 5. Assemblage final.
Ecrire un programme ou un script qui ´etant donné ne, nj et un fichier de noms d’équipes, affiche un planning des matchs en utilisant les programmes écrits dans les questions précédentes.